# Part 1 - searching for XMAS

In [92]:
import numpy as np

with open('input.txt') as f: 
    lines = f.readlines()

lines = [line.strip() for line in lines]


rows = np.array(list(lines[0]))
for line in lines[1:]: 
    row = np.array(list(line))
    rows = np.vstack((rows, row))

In [93]:
import re
regex = r"XMAS"

def horizontal_search(rows: np.ndarray, key): 
    sum_ = 0
    for row in rows: 
        joined = ''.join(row)
        sum_ += len(re.findall(key, joined))
    
    return sum_

def diagonal_search(rows: np.ndarray, key): 
    sum_ = 0
    for i in range(-(rows.shape[0]+1), (rows.shape[1]+1)): 
        joined = ''.join(np.diag(rows,i))
        # print(joined)
        sum_ += len(re.findall(key, joined))
    
    return sum_


def vertical_search(rows: np.ndarray, key): 
    return horizontal_search(rows.T, key)

def horizontal_search_reverse(rows: np.ndarray, key): 
    return horizontal_search(np.fliplr(rows), key)

def vertical_search_reverse(rows: np.ndarray, key): 
    return horizontal_search_reverse(rows.T, key)

def diagonal_search_reverse(rows: np.ndarray, key):
    return diagonal_search(np.flip(rows), key)

def anti_diagonal_search(rows: np.ndarray, key):
    return diagonal_search(np.fliplr(rows), key)

def anti_diagonal_search_reverse(rows: np.ndarray, key): 
    return anti_diagonal_search(np.flip(rows), key)

def complete_search(rows: np.ndarray, key): 
    return (horizontal_search(rows, key) + horizontal_search_reverse(rows, key) + 
            vertical_search(rows, key) + vertical_search_reverse(rows, key) + 
            diagonal_search(rows, key) + diagonal_search_reverse(rows,key) + 
            anti_diagonal_search(rows, key) + anti_diagonal_search_reverse(rows, key))


In [94]:
complete_search(rows, regex)

2401

# Part 2 - X-MAS

In [95]:
import numpy as np

with open('input.txt') as f: 
    lines = f.readlines()

lines = [line.strip() for line in lines]


rows = np.array(list(lines[0]))
for line in lines[1:]: 
    row = np.array(list(line))
    rows = np.vstack((rows, row))

Reconstruct the (row,col) coordinates from diagonal and position in the diagonal 
| Diagonal | Index | row | column |
|----------|-------|-----|--------|
| 0        | 0     | 0   | 0      |
| 0        | 1     | 1   | 1      |
| 0        | 2     | 2   | 2      |
| 1        | 0     | 0   | 1      |
| 1        | 1     | 1   | 2      |
| -1       | 0     | 1   | 0      |
| -1       | 1     | 2   | 1      |

In [96]:
def reconstruct_coordinates(diagonal_index, position_in_diagonal):
    if diagonal_index < 0: 
        starting = (abs(diagonal_index), 0)
        return (starting[0] + position_in_diagonal, starting[1] + position_in_diagonal)
    elif diagonal_index >= 0: 
        starting = (0, diagonal_index)
        return (starting[0] + position_in_diagonal, starting[1] + position_in_diagonal)
    
for testing in [(0,0),(0,1),(0,2),(1,0),(1,1),(-1,0),(-1,1)]:
    print(reconstruct_coordinates(*testing))


(0, 0)
(1, 1)
(2, 2)
(0, 1)
(1, 2)
(1, 0)
(2, 1)


In [97]:
def diagonal_search(rows: np.ndarray, key): 
    sum_ = 0
    coordinates = []
    for i in range(-(rows.shape[0]+1), (rows.shape[1]+1)): 
        joined = ''.join(np.diag(rows,i))
        # print(joined)
        matches = re.finditer(key, joined)
        for match in matches:
            central_index = match.start() + 1 # index of A letter
            coordinates.append(reconstruct_coordinates(i,central_index))
    return coordinates


def diagonal_search_reverse(rows: np.ndarray, key):
    coordinates_before_adjusting = diagonal_search(np.flip(rows), key)
    return [(rows.shape[0] - 1 - x[0], rows.shape[1] - 1 - x[1]) for x in coordinates_before_adjusting]

def anti_diagonal_search(rows: np.ndarray, key):
    coordinates_before_adjusting = diagonal_search(np.fliplr(rows), key)
    return [(x[0], rows.shape[1] - 1 - x[1]) for x in coordinates_before_adjusting]

def anti_diagonal_search_reverse(rows: np.ndarray, key): 
    coordinates_before_adjusting = anti_diagonal_search(np.flip(rows), key)
    return [(rows.shape[0] - 1 - x[0], rows.shape[1] - 1 - x[1]) for x in coordinates_before_adjusting]

import re
regex = r"MAS"

coordinates_diagonal = diagonal_search(rows, regex)
coordinates_diagonal_reverse = diagonal_search_reverse(rows,regex) 
coordinates_anti_diagonal = anti_diagonal_search(rows, regex)
coordinates_anti_diagonal_reverse = anti_diagonal_search_reverse(rows, regex)

In [99]:
len(set(coordinates_diagonal + coordinates_diagonal_reverse).intersection(set(coordinates_anti_diagonal + coordinates_anti_diagonal_reverse)))

1822

In [89]:
def print_kernel(coordinates): 
    x, y = coordinates
    print(rows[x-1:x+2, y-1:y+2])

for coordinate in coordinates_diagonal:
    print(coordinate)
    print_kernel(coordinate)

(3, 2)
[['M' 'X' 'S']
 ['S' 'A' 'M']
 ['M' 'A' 'S']]
(6, 5)
[['M' 'X' 'X']
 ['S' 'A' 'S']
 ['M' 'A' 'S']]
(1, 2)
[['M' 'M' 'S']
 ['S' 'A' 'M']
 ['M' 'X' 'S']]
(2, 6)
[['M' 'S' 'M']
 ['M' 'A' 'A']
 ['S' 'M' 'S']]
